In [2]:
import requests
from bs4 import BeautifulSoup

# Step 1: Visit the main page
main_url = 'https://www.imsnsit.org/imsnsit/'
response_main = requests.get(main_url)

if response_main.status_code == 200:
    print(f"Visited {main_url} successfully.")
    
    # Optionally parse the page to look for the student link
    soup = BeautifulSoup(response_main.text, 'html.parser')
    student_link = soup.find('a', href='student.htm')
    if student_link:
        print("Found link to student page.")
    else:
        print("Did not find student link explicitly, proceeding anyway.")
    
    # Step 2: Visit the student page
    student_url = main_url + 'student.htm'
    response_student = requests.get(student_url)
    
    if response_student.status_code == 200:
        print(f"Visited {student_url} successfully.")
        # Do something with the page, like parse content
        soup_student = BeautifulSoup(response_student.text, 'html.parser')
        print("Title of student page:", soup_student.title.string)
    else:
        print(f"Failed to visit {student_url}. Status code:", response_student.status_code)

else:
    print(f"Failed to visit {main_url}. Status code:", response_main.status_code)


Visited https://www.imsnsit.org/imsnsit/ successfully.
Found link to student page.
Visited https://www.imsnsit.org/imsnsit/student.htm successfully.
Title of student page: NSIT Delhi


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os
import time

CHROMEDRIVER_PATH = r"C:\Users\adity\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service)

try:
    driver.get("https://www.imsnsit.org/imsnsit/")
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Student Login"))
    ).click()

    # Wait for navigation
    time.sleep(2)
    print("Current URL:", driver.current_url)
    print("Page title:", driver.title)

    # Debug all <img> tags
    images = driver.find_elements(By.TAG_NAME, "img")
    print(f"Found {len(images)} image(s).")
    for img in images:
        print("IMG:", img.get_attribute("src"), "| ID:", img.get_attribute("id"))

    # Try locating the CAPTCHA image
    captcha_img = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "captchaimg"))
    )
    captcha_url = captcha_img.get_attribute("src")
    print("CAPTCHA Image URL:", captcha_url)

    # Download it
    cookies = {cookie['name']: cookie['value'] for cookie in driver.get_cookies()}
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(captcha_url, cookies=cookies, headers=headers)

    if response.status_code == 200:
        os.makedirs("captchas", exist_ok=True)
        filename = os.path.join("captchas", os.path.basename(captcha_url))
        with open(filename, "wb") as f:
            f.write(response.content)
        print("CAPTCHA saved as", filename)
    else:
        print("Failed to download CAPTCHA.")

finally:
    time.sleep(3)
    driver.quit()


Current URL: https://www.imsnsit.org/imsnsit/student.htm
Page title: NSIT Delhi
Found 0 image(s).


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7fb4fcda5+78885]
	GetHandleVerifier [0x0x7ff7fb4fce00+78976]
	(No symbol) [0x0x7ff7fb2b9bca]
	(No symbol) [0x0x7ff7fb310766]
	(No symbol) [0x0x7ff7fb310a1c]
	(No symbol) [0x0x7ff7fb364467]
	(No symbol) [0x0x7ff7fb338bcf]
	(No symbol) [0x0x7ff7fb36122f]
	(No symbol) [0x0x7ff7fb338963]
	(No symbol) [0x0x7ff7fb3016b1]
	(No symbol) [0x0x7ff7fb302443]
	GetHandleVerifier [0x0x7ff7fb7d4eed+3061101]
	GetHandleVerifier [0x0x7ff7fb7cf33d+3037629]
	GetHandleVerifier [0x0x7ff7fb7ee592+3165202]
	GetHandleVerifier [0x0x7ff7fb51730e+186766]
	GetHandleVerifier [0x0x7ff7fb51eb3f+217535]
	GetHandleVerifier [0x0x7ff7fb5059b4+114740]
	GetHandleVerifier [0x0x7ff7fb505b69+115177]
	GetHandleVerifier [0x0x7ff7fb4ec368+10728]
	BaseThreadInitThunk [0x0x7ff9ccdce8d7+23]
	RtlUserThreadStart [0x0x7ff9ce01c34c+44]


In [19]:
import requests
from bs4 import BeautifulSoup
import os

# Create a session to maintain cookies
session = requests.Session()

# Step 1: Visit the main page
main_url = "https://www.imsnsit.org/imsnsit/"
response = session.get(main_url)
response.raise_for_status()  # Check if the request was successful

# Step 2: Go to the Student Login page
student_login_url = "https://www.imsnsit.org/imsnsit/student.htm"
response = session.get(student_login_url)
response.raise_for_status()

# Step 3: Find the captcha image
soup = BeautifulSoup(response.text, 'html.parser')
captcha_img = soup.find('img', {'id': 'captchaimg'})

if captcha_img and 'src' in captcha_img.attrs:
    # Step 4: Download the captcha image
    captcha_src = captcha_img['src']
    captcha_url = f"https://www.imsnsit.org/imsnsit/{captcha_src}"
    
    # Get the image data
    image_response = session.get(captcha_url)
    image_response.raise_for_status()
    
    # Save the image
    image_name = os.path.basename(captcha_src)
    with open(image_name, 'wb') as f:
        f.write(image_response.content)
    
    print(f"Captcha image saved as {image_name}")
else:
    print("Could not find captcha image on the page")

Could not find captcha image on the page


In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests

# Setup Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
CHROMEDRIVER_PATH = r"C:\Users\adity\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service)

# Initialize the driver
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Step 1: Visit main page
    driver.get("https://www.imsnsit.org/imsnsit/")
    time.sleep(2)  # Wait for initial load

    # Step 2: Find and click Student Login link
    student_login = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//a[@href='student.htm']"))
    )
    student_login.click()
    
    # Step 3: Wait for CAPTCHA to load
    captcha_img = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "captchaimg"))
    )
    
    # Step 4: Get CAPTCHA URL
    captcha_src = captcha_img.get_attribute("src")
    if not captcha_src:
        raise Exception("CAPTCHA src attribute not found")
    
    # Step 5: Download the image
    cookies = {c['name']: c['value'] for c in driver.get_cookies()}
    captcha_url = f"https://www.imsnsit.org/imsnsit/{captcha_src}"
    
    response = requests.get(captcha_url, cookies=cookies)
    response.raise_for_status()
    
    with open('captcha.jpg', 'wb') as f:
        f.write(response.content)
    
    print("CAPTCHA image downloaded successfully as captcha.jpg")

except Exception as e:
    print(f"Error: {str(e)}")
    # Take screenshot for debugging
    driver.save_screenshot('error_screenshot.png')
    print("Screenshot saved as error_screenshot.png")

finally:
    driver.quit()

Error: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7fb4fcda5+78885]
	GetHandleVerifier [0x0x7ff7fb4fce00+78976]
	(No symbol) [0x0x7ff7fb2b9bca]
	(No symbol) [0x0x7ff7fb310766]
	(No symbol) [0x0x7ff7fb310a1c]
	(No symbol) [0x0x7ff7fb364467]
	(No symbol) [0x0x7ff7fb338bcf]
	(No symbol) [0x0x7ff7fb36122f]
	(No symbol) [0x0x7ff7fb338963]
	(No symbol) [0x0x7ff7fb3016b1]
	(No symbol) [0x0x7ff7fb302443]
	GetHandleVerifier [0x0x7ff7fb7d4eed+3061101]
	GetHandleVerifier [0x0x7ff7fb7cf33d+3037629]
	GetHandleVerifier [0x0x7ff7fb7ee592+3165202]
	GetHandleVerifier [0x0x7ff7fb51730e+186766]
	GetHandleVerifier [0x0x7ff7fb51eb3f+217535]
	GetHandleVerifier [0x0x7ff7fb5059b4+114740]
	GetHandleVerifier [0x0x7ff7fb505b69+115177]
	GetHandleVerifier [0x0x7ff7fb4ec368+10728]
	BaseThreadInitThunk [0x0x7ff9ccdce8d7+23]
	RtlUserThreadStart [0x0x7ff9ce01c34c+44]

Screenshot saved as error_screenshot.png


In [24]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
from urllib.parse import urljoin

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in background
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set realistic user agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
chrome_options.add_argument(f'user-agent={user_agent}')

# Initialize WebDriver
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

try:
    # Step 1: Visit main page with Selenium to establish session
    main_url = "https://www.imsnsit.org/imsnsit/"
    driver.get(main_url)
    time.sleep(2)  # Allow page to load

    # Step 2: Extract cookies from Selenium and prepare for requests session
    selenium_cookies = driver.get_cookies()
    session = requests.Session()
    
    # Convert Selenium cookies to requests cookies
    for cookie in selenium_cookies:
        session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain'])
    
    # Set realistic headers
    headers = {
        'User-Agent': user_agent,
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': main_url,
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
    session.headers.update(headers)

    # Step 3: Navigate to student page with Selenium (JavaScript required)
    student_link = driver.find_element("xpath", "//a[@href='student.htm']")
    student_link.click()
    time.sleep(3)  # Wait for page and CAPTCHA to load

    # Step 4: Get current URL and CAPTCHA src
    student_page_url = driver.current_url
    captcha_img = driver.find_element("id", "captchaimg")
    captcha_src = captcha_img.get_attribute("src")
    
    if not captcha_src:
        raise Exception("CAPTCHA source not found")

    # Step 5: Download CAPTCHA using requests session (with maintained cookies)
    captcha_url = urljoin(student_page_url, captcha_src)
    
    # Add referer header just for the CAPTCHA download
    captcha_headers = {'Referer': student_page_url}
    response = session.get(captcha_url, headers=captcha_headers)
    response.raise_for_status()

    # Save the image
    with open('captcha.jpg', 'wb') as f:
        f.write(response.content)
    
    print("CAPTCHA successfully downloaded as captcha.jpg")

except Exception as e:
    print(f"Error occurred: {str(e)}")
    # Save screenshot for debugging
    driver.save_screenshot('error.png')
    print("Screenshot saved as error.png")

finally:
    driver.quit()

ModuleNotFoundError: No module named 'webdriver_manager'

In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Configure Chrome
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

try:
    driver.get("https://www.imsnsit.org/imsnsit/")
    driver.find_element("xpath", "//a[@href='student.htm']").click()
    
    # Wait for CAPTCHA to load
    captcha = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "captchaimg"))
    )
    
    # Get CAPTCHA as PNG screenshot
    captcha.screenshot('captcha.png')
    print("CAPTCHA saved as captcha.png")

finally:
    driver.quit()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\adity\.conda\envs\captchaenv\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\adity\AppData\Local\Temp\ipykernel_26396\3600133554.py", line 12, in <module>
    service=Service(ChromeDriverManager().install()),
  File "c:\Users\adity\.conda\envs\captchaenv\lib\site-packages\webdriver_manager\chrome.py", line 40, in install
    driver_path = self._get_driver_binary_path(self.driver)
  File "c:\Users\adity\.conda\envs\captchaenv\lib\site-packages\webdriver_manager\core\manager.py", line 41, in _get_driver_binary_path
    binary_path = self._cache_manager.save_file_to_cache(driver, file)
  File "c:\Users\adity\.conda\envs\captchaenv\lib\site-packages\webdriver_manager\core\driver_cache.py", line 52, in save_file_to_cache
    path = self.__get_path(driver)
  File "c:\Users\adity\.conda\envs\captchaenv\lib\site-packages\webdriver_manager\core\d

In [26]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
from urllib.parse import urljoin

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in background
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set realistic user agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
chrome_options.add_argument(f'user-agent={user_agent}')

# Initialize WebDriver
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

try:
    # Step 1: Visit main page with Selenium to establish session
    main_url = "https://www.imsnsit.org/imsnsit/"
    driver.get(main_url)
    time.sleep(2)  # Allow page to load

    # Step 2: Extract cookies from Selenium and prepare for requests session
    selenium_cookies = driver.get_cookies()
    session = requests.Session()
    
    # Convert Selenium cookies to requests cookies
    for cookie in selenium_cookies:
        session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain'])
    
    # Set realistic headers
    headers = {
        'User-Agent': user_agent,
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': main_url,
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
    session.headers.update(headers)

    # Step 3: Navigate to student page with Selenium (JavaScript required)
    student_link = driver.find_element("xpath", "//a[@href='student.htm']")
    student_link.click()
    time.sleep(3)  # Wait for page and CAPTCHA to load

    # Step 4: Get current URL and CAPTCHA src
    student_page_url = driver.current_url
    captcha_img = driver.find_element("id", "captchaimg")
    captcha_src = captcha_img.get_attribute("src")
    
    if not captcha_src:
        raise Exception("CAPTCHA source not found")

    # Step 5: Download CAPTCHA using requests session (with maintained cookies)
    captcha_url = urljoin(student_page_url, captcha_src)
    
    # Add referer header just for the CAPTCHA download
    captcha_headers = {'Referer': student_page_url}
    response = session.get(captcha_url, headers=captcha_headers)
    response.raise_for_status()

    # Save the image
    with open('captcha.jpg', 'wb') as f:
        f.write(response.content)
    
    print("CAPTCHA successfully downloaded as captcha.jpg")

except Exception as e:
    print(f"Error occurred: {str(e)}")
    # Save screenshot for debugging
    driver.save_screenshot('error.png')
    print("Screenshot saved as error.png")

finally:
    driver.quit()

Error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="captchaimg"]"}
  (Session info: chrome=137.0.7151.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0xe23b03+62899]
	GetHandleVerifier [0x0xe23b44+62964]
	(No symbol) [0x0xc510f3]
	(No symbol) [0x0xc9980e]
	(No symbol) [0x0xc99bab]
	(No symbol) [0x0xce25c2]
	(No symbol) [0x0xcbe554]
	(No symbol) [0x0xcdfd81]
	(No symbol) [0x0xcbe306]
	(No symbol) [0x0xc8d670]
	(No symbol) [0x0xc8e4e4]
	GetHandleVerifier [0x0x1084793+2556483]
	GetHandleVerifier [0x0x107fd02+2537394]
	GetHandleVerifier [0x0xe4a2fa+220586]
	GetHandleVerifier [0x0xe3aae8+157080]
	GetHandleVerifier [0x0xe4141d+184013]
	GetHandleVerifier [0x0xe2ba68+95512]
	GetHandleVerifier [0x0xe2bc10+95936]
	GetHandleVerifier [0x0xe16b5a+9738]
	BaseThreadInitThunk [0x0x76ca5d49+25]
	RtlInitializeEx

In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
import time
from urllib.parse import urljoin

# Configure Chrome with anti-detection measures
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

# Set realistic user agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
chrome_options.add_argument(f'user-agent={user_agent}')

# Initialize WebDriver
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

try:
    # Step 1: Visit main page
    main_url = "https://www.imsnsit.org/imsnsit/"
    driver.get(main_url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    # Debug: Save screenshot
    driver.save_screenshot('step1_main_page.png')

    # Step 2: Click Student Login
    student_link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@href='student.htm']"))
    )
    student_link.click()
    
    # Step 3: Wait for CAPTCHA - first check if in iframe
    time.sleep(2)  # Brief pause for iframe loading
    iframes = driver.find_elements(By.TAG_NAME, "iframe")
    if iframes:
        print(f"Found {len(iframes)} iframes, switching...")
        driver.switch_to.frame(iframes[0])
    
    # Wait for CAPTCHA with multiple fallbacks
    try:
        captcha_img = WebDriverWait(driver, 15).until(
            EC.visibility_of_element_located((By.ID, "captchaimg"))
        )
    except:
        # Try alternative selectors if ID fails
        captcha_img = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "img[id*='captcha']"))
        )
    
    # Debug: Save screenshot
    driver.save_screenshot('step3_captcha_found.png')
    
    # Get CAPTCHA URL
    captcha_src = captcha_img.get_attribute("src")
    if not captcha_src:
        raise Exception("CAPTCHA src attribute empty")
    
    # Step 4: Download with requests session
    session = requests.Session()
    for cookie in driver.get_cookies():
        session.cookies.set(cookie['name'], cookie['value'])
    
    headers = {
        'User-Agent': user_agent,
        'Referer': driver.current_url
    }
    
    captcha_url = urljoin(driver.current_url, captcha_src)
    response = session.get(captcha_url, headers=headers)
    response.raise_for_status()
    
    with open('captcha.jpg', 'wb') as f:
        f.write(response.content)
    
    print("Successfully downloaded CAPTCHA")

except Exception as e:
    print(f"Error: {str(e)}")
    driver.save_screenshot('error_debug.png')
    print("Saved error_debug.png for inspection")

finally:
    driver.quit()

Error: Message: 
Stacktrace:
	GetHandleVerifier [0x0xe23b03+62899]
	GetHandleVerifier [0x0xe23b44+62964]
	(No symbol) [0x0xc510f3]
	(No symbol) [0x0xc9980e]
	(No symbol) [0x0xc99bab]
	(No symbol) [0x0xce25c2]
	(No symbol) [0x0xcbe554]
	(No symbol) [0x0xcdfd81]
	(No symbol) [0x0xcbe306]
	(No symbol) [0x0xc8d670]
	(No symbol) [0x0xc8e4e4]
	GetHandleVerifier [0x0x1084793+2556483]
	GetHandleVerifier [0x0x107fd02+2537394]
	GetHandleVerifier [0x0xe4a2fa+220586]
	GetHandleVerifier [0x0xe3aae8+157080]
	GetHandleVerifier [0x0xe4141d+184013]
	GetHandleVerifier [0x0xe2ba68+95512]
	GetHandleVerifier [0x0xe2bc10+95936]
	GetHandleVerifier [0x0xe16b5a+9738]
	BaseThreadInitThunk [0x0x76ca5d49+25]
	RtlInitializeExceptionChain [0x0x77d3d09b+107]
	RtlGetAppContainerNamedObjectPath [0x0x77d3d021+561]

Saved error_debug.png for inspection


In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import os
import requests
import re

def selenium_captcha_download(url, output_folder='captchas'):
    # Configure Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    # Initialize WebDriver
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )
    
    try:
        # Create output directory
        os.makedirs(output_folder, exist_ok=True)
        
        # Navigate to page
        driver.get(url)
        
        # Find all CAPTCHA images
        captcha_images = driver.find_elements("xpath", "//img[contains(@src, 'captcha')]")
        
        if not captcha_images:
            print("No CAPTCHA images found on the page")
            return
        
        print(f"Found {len(captcha_images)} CAPTCHA image(s)")
        
        # Get cookies for session
        cookies = {c['name']: c['value'] for c in driver.get_cookies()}
        
        # Download each CAPTCHA
        for img in captcha_images:
            img_url = img.get_attribute("src")
            if not img_url.startswith('http'):
                img_url = url + img_url if not img_url.startswith('/') else f"https://{url.split('/')[2]}{img_url}"
            
            filename = os.path.join(output_folder, os.path.basename(img_url))
            
            # Download with session cookies
            response = requests.get(img_url, cookies=cookies)
            response.raise_for_status()
            
            with open(filename, 'wb') as f:
                f.write(response.content)
            
            print(f"Downloaded: {filename}")
            
    except Exception as e:
        print(f"Error: {str(e)}")
    finally:
        driver.quit()

# Usage
selenium_captcha_download('https://www.imsnsit.org/imsnsit/student.htm')

No CAPTCHA images found on the page


In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import os
import time
import requests
from urllib.parse import urljoin

def save_complete_page_with_selenium(url, output_dir):
    # Configure Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    # Initialize WebDriver
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )
    
    try:
        # Create directories
        os.makedirs(output_dir, exist_ok=True)
        captcha_dir = os.path.join(output_dir, 'captchas')
        os.makedirs(captcha_dir, exist_ok=True)
        
        # Navigate to page
        driver.get(url)
        time.sleep(3)  # Wait for page to load
        
        # Save complete page source
        page_source = driver.page_source
        with open(os.path.join(output_dir, 'page.html'), 'w', encoding='utf-8') as f:
            f.write(page_source)
        
        # Find all images
        images = driver.find_elements("tag name", "img")
        
        # Download each image
        for img in images:
            img_url = img.get_attribute("src")
            if img_url and 'captcha' in img_url.lower():
                try:
                    # Handle relative URLs
                    full_url = urljoin(url, img_url)
                    
                    # Get filename
                    filename = os.path.basename(img_url)
                    filepath = os.path.join(captcha_dir, filename)
                    
                    # Download image
                    response = requests.get(full_url)
                    response.raise_for_status()
                    
                    with open(filepath, 'wb') as f:
                        f.write(response.content)
                    
                    print(f"Saved CAPTCHA: {filepath}")
                    
                except Exception as e:
                    print(f"Failed to download {img_url}: {str(e)}")
        
    finally:
        driver.quit()

# Usage
save_complete_page_with_selenium("https://www.imsnsit.org/imsnsit/student.htm", "selenium_download")

In [34]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def download_captcha_files():
    # Configure Chrome
    chrome_options = Options()
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    # Uncomment for headless mode
    # chrome_options.add_argument("--headless")

    # Initialize WebDriver
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )

    try:
        # Step 1: Visit main page (just to establish session)
        main_url = "https://www.imsnsit.org/imsnsit/"
        print(f"Visiting main page: {main_url}")
        driver.get(main_url)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

        # Step 2: Navigate to student login page
        student_url = "https://www.imsnsit.org/imsnsit/student.htm"
        print(f"Navigating to student page: {student_url}")
        driver.get(student_url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

        # Step 3: Get all image elements
        images = driver.find_elements(By.TAG_NAME, "img")
        print(f"Found {len(images)} images on the page")

        # Create output directory
        os.makedirs("captchas", exist_ok=True)

        # Step 4: Download only captcha files
        captcha_count = 0
        for img in images:
            src = img.get_attribute("src")
            if src and src.lower().split('/')[-1].startswith("captcha"):
                try:
                    # Build absolute URL
                    captcha_url = urljoin(student_url, src)
                    filename = src.split('/')[-1]
                    filepath = os.path.join("captchas", filename)

                    # Download using requests with same cookies
                    session = requests.Session()
                    for cookie in driver.get_cookies():
                        session.cookies.set(cookie['name'], cookie['value'])

                    response = session.get(captcha_url)
                    response.raise_for_status()

                    with open(filepath, 'wb') as f:
                        f.write(response.content)

                    print(f"Downloaded: {filepath}")
                    captcha_count += 1

                except Exception as e:
                    print(f"Failed to download {src}: {str(e)}")

        print(f"\nDownloaded {captcha_count} CAPTCHA files")

    except Exception as e:
        print(f"Error: {str(e)}")
        driver.save_screenshot('error.png')
        print("Saved error screenshot: error.png")

    finally:
        driver.quit()

if __name__ == "__main__":
    download_captcha_files()

Visiting main page: https://www.imsnsit.org/imsnsit/
Navigating to student page: https://www.imsnsit.org/imsnsit/student.htm
Error: Message: 
Stacktrace:
	GetHandleVerifier [0x0xe23b03+62899]
	GetHandleVerifier [0x0xe23b44+62964]
	(No symbol) [0x0xc510f3]
	(No symbol) [0x0xc9980e]
	(No symbol) [0x0xc99bab]
	(No symbol) [0x0xce25c2]
	(No symbol) [0x0xcbe554]
	(No symbol) [0x0xcdfd81]
	(No symbol) [0x0xcbe306]
	(No symbol) [0x0xc8d670]
	(No symbol) [0x0xc8e4e4]
	GetHandleVerifier [0x0x1084793+2556483]
	GetHandleVerifier [0x0x107fd02+2537394]
	GetHandleVerifier [0x0xe4a2fa+220586]
	GetHandleVerifier [0x0xe3aae8+157080]
	GetHandleVerifier [0x0xe4141d+184013]
	GetHandleVerifier [0x0xe2ba68+95512]
	GetHandleVerifier [0x0xe2bc10+95936]
	GetHandleVerifier [0x0xe16b5a+9738]
	BaseThreadInitThunk [0x0x76ca5d49+25]
	RtlInitializeExceptionChain [0x0x77d3d09b+107]
	RtlGetAppContainerNamedObjectPath [0x0x77d3d021+561]

Saved error screenshot: error.png


In [41]:
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

class CaptchaDownloader:
    def __init__(self):
        self.session = requests.Session()
        self._configure_session()
        
    def _configure_session(self):
        """Configure session with headers and cookies similar to Kotlin implementation"""
        headers = {
            'User-Agent': 'Mozilla/5.0 (Linux; Android 9; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.74 Mobile Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Referer': 'https://imsnsit.org/imsnsit/',
            'Sec-Fetch-User': '?1'
        }
        self.session.headers.update(headers)
        
    def initialize_session(self):
        """Initialize session by visiting required pages"""
        # Visit main page to establish session
        self.session.get('https://imsnsit.org/imsnsit/')
        
        # Visit intermediate page (similar to student_login110.php in Kotlin)
        self.session.get('https://www.imsnsit.org/imsnsit/student_login110.php')
        
    def download_captcha(self, output_dir='captchas'):
        """Download CAPTCHA image following the same flow as Kotlin code"""
        try:
            # Create output directory if it doesn't exist
            os.makedirs(output_dir, exist_ok=True)
            
            # 1. Access student login page
            login_url = 'https://www.imsnsit.org/imsnsit/student_login.php'
            response = self.session.get(login_url)
            response.raise_for_status()
            
            # 2. Parse HTML to find CAPTCHA image
            soup = BeautifulSoup(response.text, 'html.parser')
            captcha_img = soup.find('img', {'id': 'captchaimg'})
            
            if not captcha_img or not captcha_img.get('src'):
                raise ValueError("CAPTCHA image element not found in page")
            
            # 3. Build absolute CAPTCHA URL
            captcha_relative_url = captcha_img['src']
            captcha_url = urljoin(login_url, captcha_relative_url)
            
            # 4. Download CAPTCHA image with referer header
            captcha_response = self.session.get(
                captcha_url,
                headers={'Referer': login_url}
            )
            captcha_response.raise_for_status()
            
            # 5. Save CAPTCHA image
            filename = os.path.basename(captcha_relative_url)
            output_path = os.path.join(output_dir, filename)
            
            with open(output_path, 'wb') as f:
                f.write(captcha_response.content)
            
            print(f"Successfully downloaded CAPTCHA to: {output_path}")
            return output_path
            
        except Exception as e:
            print(f"Error downloading CAPTCHA: {str(e)}")
            return None

    def get_login_page_data(self):
        """Get additional login page data like in Kotlin implementation"""
        try:
            login_url = 'https://www.imsnsit.org/imsnsit/student_login.php'
            response = self.session.get(login_url)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract additional form fields
            hrand_num = soup.find('input', {'id': 'HRAND_NUM'})['value'] if soup.find('input', {'id': 'HRAND_NUM'}) else ''
            fy = soup.find('input', {'id': 'fy'})['value'] if soup.find('input', {'id': 'fy'}) else ''
            comp = soup.find('input', {'id': 'comp'})['value'] if soup.find('input', {'id': 'comp'}) else ''
            
            # Extract form action
            form = soup.find('form', {'name': 'f1'})
            login_action = urljoin(login_url, form['action']) if form else ''
            
            return {
                'hrand_num': hrand_num,
                'fy': fy,
                'comp': comp,
                'login_action': login_action
            }
            
        except Exception as e:
            print(f"Error getting login page data: {str(e)}")
            return None

# Usage Example
if __name__ == "__main__":
    downloader = CaptchaDownloader()
    
    # Initialize session (same as Kotlin flow)
    downloader.initialize_session()
    
    # Download CAPTCHA
    captcha_path = downloader.download_captcha()
    
    # Get additional login data (optional)
    login_data = downloader.get_login_page_data()
    if login_data:
        print("Additional login data:", login_data)

Successfully downloaded CAPTCHA to: captchas\captcha_1750587434.jpg
Additional login data: {'hrand_num': '19547', 'fy': '2025-26', 'comp': 'NETAJI SUBHAS UNIVERSITY OF TECHNOLOGY', 'login_action': 'https://www.imsnsit.org/imsnsit/student_login.php'}


In [ ]:
import os
import random
import time
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

class CaptchaAutomator:
    def __init__(self):
        self.session = requests.Session()
        self.download_count = 500
        self.max_captchas = 700
        self.output_dir = "captchas"
        self.user_agents = [
            # Chrome
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
            
            # Firefox
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:126.0) Gecko/20100101 Firefox/126.0",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:126.0) Gecko/20100101 Firefox/126.0",
            
            # Safari
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4 Safari/605.1.15",
            
            # Mobile
            "Mozilla/5.0 (Linux; Android 10; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6422.165 Mobile Safari/537.36",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 17_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4 Mobile/15E148 Safari/604.1"
        ]
        self._configure_session()
        
    def _configure_session(self):
        """Configure session with headers and cookies"""
        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Referer': 'https://imsnsit.org/imsnsit/',
            'Sec-Fetch-User': '?1',
            'User-Agent': random.choice(self.user_agents)
        }
        self.session.headers.update(headers)
        
    def _random_delay(self):
        """Wait for a random interval between 5-15 seconds"""
        delay = random.uniform(5, 15)
        print(f"Waiting {delay:.2f} seconds before next download...")
        time.sleep(delay)
        
    def _rotate_user_agent(self):
        """Change user agent randomly"""
        self.session.headers.update({
            'User-Agent': random.choice(self.user_agents)
        })
        
    def initialize_session(self):
        """Initialize session by visiting required pages"""
        try:
            # Visit main page to establish session
            self.session.get('https://imsnsit.org/imsnsit/')
            
            # Visit intermediate page
            self.session.get('https://www.imsnsit.org/imsnsit/student_login110.php')
            
            # Random delay before continuing
            self._random_delay()
            
        except Exception as e:
            print(f"Session initialization failed: {str(e)}")
            raise

    def download_captcha(self):
        """Download a single CAPTCHA image"""
        try:
            # Rotate user agent for this request
            self._rotate_user_agent()
            
            # Access student login page
            login_url = 'https://www.imsnsit.org/imsnsit/student_login.php'
            response = self.session.get(login_url)
            response.raise_for_status()
            
            # Parse HTML to find CAPTCHA image
            soup = BeautifulSoup(response.text, 'html.parser')
            captcha_img = soup.find('img', {'id': 'captchaimg'})
            
            if not captcha_img or not captcha_img.get('src'):
                raise ValueError("CAPTCHA image element not found in page")
            
            # Build absolute CAPTCHA URL
            captcha_relative_url = captcha_img['src']
            captcha_url = urljoin(login_url, captcha_relative_url)
            
            # Download CAPTCHA image
            captcha_response = self.session.get(
                captcha_url,
                headers={'Referer': login_url}
            )
            captcha_response.raise_for_status()
            
            # Generate unique filename
            timestamp = int(time.time())
            filename = f"captcha_{timestamp}_{self.download_count}.jpg"
            output_path = os.path.join(self.output_dir, filename)
            
            # Save CAPTCHA image
            with open(output_path, 'wb') as f:
                f.write(captcha_response.content)
            
            self.download_count += 1
            print(f"Downloaded CAPTCHA #{self.download_count} to: {output_path}")
            
            # Random delay after successful download
            self._random_delay()
            
            return True
            
        except Exception as e:
            print(f"Error downloading CAPTCHA: {str(e)}")
            return False

    def run(self):
        """Main automation loop"""
        # Create output directory
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Initial session setup
        self.initialize_session()
        
        # Main download loop
        while self.download_count < self.max_captchas:
            try:
                # Reinitialize session periodically (every 20 downloads)
                if self.download_count % 20 == 0:
                    self.initialize_session()
                
                # Attempt to download CAPTCHA
                success = self.download_captcha()
                
                if not success:
                    # If failed, wait longer before retry
                    time.sleep(random.uniform(15, 30))
                    self.initialize_session()
                    
            except KeyboardInterrupt:
                print("\nStopping automation...")
                break
            except Exception as e:
                print(f"Unexpected error: {str(e)}")
                time.sleep(30)
                self.initialize_session()
        
        print(f"\nCompleted downloading {self.download_count} CAPTCHAs")

if __name__ == "__main__":
    automator = CaptchaAutomator()
    automator.run()

Waiting 6.72 seconds before next download...
Waiting 11.09 seconds before next download...
Error downloading CAPTCHA: CAPTCHA image element not found in page
Waiting 5.21 seconds before next download...
Waiting 13.81 seconds before next download...
Downloaded CAPTCHA #501 to: captchas\captcha_1750627882_500.jpg
Waiting 12.63 seconds before next download...
Downloaded CAPTCHA #502 to: captchas\captcha_1750627895_501.jpg
Waiting 7.68 seconds before next download...
Downloaded CAPTCHA #503 to: captchas\captcha_1750627903_502.jpg
Waiting 5.37 seconds before next download...
Downloaded CAPTCHA #504 to: captchas\captcha_1750627909_503.jpg
Waiting 7.73 seconds before next download...
Error downloading CAPTCHA: CAPTCHA image element not found in page
Waiting 11.10 seconds before next download...
Downloaded CAPTCHA #505 to: captchas\captcha_1750627944_504.jpg
Waiting 14.79 seconds before next download...
Downloaded CAPTCHA #506 to: captchas\captcha_1750627959_505.jpg
Waiting 12.14 seconds befor